In [ ]:
!pip install -q -U google-generativeai
!pip install openai

In [ ]:
color_input = "banana-crepe"
import json
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import Image, display, HTML
from IPython.display import Markdown

from google.colab import userdata

from openai import OpenAI

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))



GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')


client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

with open('/content/pantone-colors-by-key.json') as f:
    pantone = json.load(f)
print(pantone)

In [ ]:
color = pantone[color_input]
print(color)



In [ ]:
prompt = f"'Little {color['Name']} Riding Hood' is a spin off picture book story of 'Little Red Riding Hood' where the main character's color is called {color['Name']} instead of Red, and has the HEX code {color['HexCode']}. The story is written for kids. Create a script for this picture book, following the previous description. Return a list of paragraphs, each corresponding to one page of the picture book. Be creative, the story is based on 'Little Red Riding Hood’, but the setting of the story can be anywhere, it doesn’t have to be in a forest, choose an appropriate setting based on the color name."
prompt

In [ ]:
response = model.generate_content(prompt)

# print("here is some feedback on your prompt")
# print(response.prompt_feedback)

to_markdown(response.text)


In [ ]:
pages = []
for content in response.text.split("\n"):
    if len(content)>0 and "*" not in content:
        pages.append(content)
print(pages)

In [ ]:
def generate_image(page):
    generate_prompt = f"Imagine you are a student that needs to create an image for a picture book. Write a text that you can use later as an input to generate an image using an AI image generator. The prompt you write should follow these guideilines: The image should be in the style of a children's picture book illustration. The scene should feature a character named '{color['Name']} Riding Hood,' wearing a vibrant hooded cloak in the specific HEX color {color['HexCode']}. The scene that the image refers to is: '{page}.’ The illustration should have bold outlines, a textured appearance, and a whimsical charm, emulating the look of hand-drawn art with colored pencils and watercolors."
    # print(generate_prompt)
    image_prompt = model.generate_content(generate_prompt).text
    print("PROMPT GENERATED")
    # print(response.text)
    to_markdown(image_prompt)
    response = client.images.generate(
        model="dall-e-3",
        prompt=image_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )
    image_url = response.data[0].url
    print(f"Image URL: {image_url}")
    display(Image(url=image_url, width=500))

generate_image(pages[0])

In [ ]:
for page in pages:
    print(page)
    generate_image(page)
